# Assignment 2: Decision Trees

In this assignment, you are going to implement a decision tree (or random forest) to forcast the weather.

## Description

- You must implement a `Model` class for training and predicting:
  ```python
  X, y = load_dataset()

  model = Model(num_features, num_classes)

  # training
  model.fit(X, y)

  # prediction
  y_pred = model.predict(X)
  ```
- Please search (Ctrl+F) for `TODO` to see what you need to do.
- About the dataset
  - Given the **training set**, please **train/validate** on it.  
  (note that your model will get bad testing score if it overfits on the training set)
  - After submitting the assignment, we will train on the same training set and test on the hidden **testing set** for scoring (using [f1-score](https://towardsdatascience.com/a-look-at-precision-recall-and-f1-score-36b5fd0dd3ec#11b8)).

### Typical performance

- **Random Guess**  
  F1-Score: 0.30  
  Accuracy: 0.50
- **Always Predict 1**  
  F1-Score: 0.37  
  Accuracy: 0.22
- **Always Predict 0**  
  F1-Score: 0.00  
  Accuracy: 0.77
- **sklearn.tree.DecisionTreeClassifier**  
  - **Training (5-fold cross-validation mean)**  
    F1-Score: 0.63-0.99  
    Accuracy: 0.85-0.99
  - **Validation (5-fold cross-validation mean)**  
    F1-Score: 0.50-0.60  
    Accuracy: 0.75-0.90


In [5]:
###########################
# DO NOT CHANGE THIS CELL #
###########################

import os
import urllib.request
import numpy as np
import pandas as pd
from typing import Tuple
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score, accuracy_score


def load_dataset(url):
  """ Get and load weather dataset. """

  path = url.split('/')[-1] # get the file name from url

  if not os.path.exists(path):
    print('Download:', url)
    urllib.request.urlretrieve(url, path)

  return pd.read_pickle(path) # pickle protocol=4


def get_input_target(df):
  """ Get X and y from weather dataset. """
  
  target_column = 'RainTomorrow' # predict 1 if it rains tomorrow

  X = df.drop(columns=[target_column]).to_numpy() # drop means to abandon target column, to_numpy means to change to an array, x是除了RainTomorrow的多維陣列
  y = df[target_column].to_numpy() # to_numpy means to change to an array, y是橫的一維陣列 [0, 0, 0, 0, 0]

  return X, y


def k_fold_cv(model_create_fn, X, y, k=5):
  """ Run k-fold cross-validation. """

  results = []

  idxs = list(range(X.shape[0])) # .shape[0] 回傳第0列的大小 ref: https://www.itread01.com/content/1540479380.html
                   # ref: https://blog.csdn.net/qq_28618765/article/details/78081959
                   # list(range(10)) = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
  np.random.shuffle(idxs)

  for i, (train_idxs, val_idxs) in enumerate(KFold(k).split(idxs)):
    splits = {'train': (X[train_idxs], y[train_idxs]),
              'val':   (X[val_idxs],   y[val_idxs]  )}

    print('Run {}:'.format(i+1))

    model = model_create_fn()
    model.fit(*splits['train']) # training

    for name, (X_split, y_split) in splits.items():
      y_pred = model.predict(X_split)

      result = {'split': name,
                'f1': f1_score(y_pred, y_split),
                'acc': accuracy_score(y_pred, y_split)}
      results.append(result)

      print('{split:>8s}: f1={f1:.4f} acc={acc:.4f}'.format(**result))

  return pd.DataFrame(results)


# @begin

In [6]:
# TODO: you can define or import something here (optional)

class Model: # = DecisionTreeClassifier()

  def __init__(self, num_features: int, num_classes: int):
    """
    Initialize the model.

    Args:
        num_features (int) : the input feature size.
        num_classes (int) : number of output classes.
    """

    self.num_features = num_features
    self.num_classes = num_classes

    # TODO: implement your model initialization here (optional)
    self.tree = None

  def fit(self, X: np.ndarray, y: np.ndarray):
    """
    Train on input/target pairs.

    Args:
        X (np.ndarray) : training inputs with shape (num_inputs, num_features).
        y (np.ndarray) : training targets with shape (num_inputs,).
    """

    # TODO: implement your training algorithm here
    num_inputs = X.shape[0]
    y.reshape(num_inputs, 1)
    train = np.concatenate((X,y[:,None]), axis=1) # combining X and y into one train array. ref: https://stackoverflow.com/questions/30305069/numpy-concatenate-2d-arrays-with-1d-array
    self.tree = build_tree(train, max_depth = 2, min_size = num_inputs/(2**2)) # modify max_depth and min_size for different decision tree


  def predict(self, X: np.ndarray) -> np.ndarray:
    '''
    Predict y given X.

    Args:
        X (np.ndarray) : inputs, shape: (num_inputs, num_features).
    
    Returns:
        np.ndarray : the predicted integer outputs, shape: (num_inputs,).
    '''

    # TODO: implement your prediction algorithm here
    predictions = list()
    for row in X:
      prediction = predict_(self.tree, row)
      predictions.append(prediction)
    return(predictions)

# ref: https://www.youtube.com/watch?v=-W0DnxQK1Eo > concept for decision tree
# ref: https://www.youtube.com/watch?v=tfGuRVTwxq8 > python code for using sklearn to implment decision tree
# ref: https://gist.github.com/djosix/e1e86fc91fe60cff86ce0626b9aa020b > TA's example code to fasten the split function
# ref: https://machinelearningmastery.com/implement-decision-tree-algorithm-scratch-python/ > how to implement code from scratch

# Calculate the Gini index for a split dataset
def gini_index(groups, classes):
  '''
  input ex:
  groups = [[[1, 1], [1, 0]], [[1, 1], [1, 0]]]
  classes = [0, 1] 

  '''
	# count all samples at split point
  num_instances = float(sum([len(group) for group in groups]))
	# sum weighted Gini index for each group
  gini = 0.0
  for group in groups:
    size = float(len(group))
    # avoid divide by zero
    if size == 0:
      continue
    score = 0.0
    # score the group based on the score for each class
    for class_val in classes:
      p = [row[-1] for row in group].count(class_val) / size # list.count(X) means to count how many times X appears in the list
                                        # ref: https://www.guru99.com/python-list-count.html
      score += p * p
    # weight the group score by its relative size
    gini += (1.0 - score) * (size / num_instances)
  return gini

# Split a dataset based on an attribute and an attribute value
def test_split(index, value, dataset):
  low_idxs = dataset[:, index] <= value
  left = dataset[low_idxs]
  right = dataset[~low_idxs]
  return left, right
  
# Select the best split point for a dataset
def get_split(dataset):
  #class_values = list(set(row[-1] for row in dataset)) # set() ref: https://www.runoob.com/python/python-func-set.html
  class_values = list([0, 1])
  b_index, b_value, b_score, b_groups = 999, 999, 999, None
  dataset = np.array(dataset) # ref: https://stackoverflow.com/questions/15884527/how-can-i-prevent-the-typeerror-list-indices-must-be-integers-not-tuple-when-c
  for index in range(len(dataset[0])-1):
    ds = dataset[:, index] # 取第index個feature的值
    ds = np.unique(ds) # 將array轉為1D並刪除重複值
    ds = (ds[1:] + ds[:-1]) / 2

    row_num = len(dataset)
    dataset_ = dataset[:int(0.35*row_num), :] # only calculate with the first 0.35 of the dataset, too long to test all datas

    for val in ds:
      groups = test_split(index, val, dataset_)
      gini = gini_index(groups, class_values)
      if gini < b_score:
        b_index, b_value, b_score, b_groups = index, val, gini, groups
  b_groups = [group.tolist() for group in b_groups]
  return {'index':b_index, 'value':b_value, 'groups':b_groups}

# Create a terminal node value
def to_terminal(group):
  outcomes = [row[-1] for row in group]
  return max(set(outcomes), key=outcomes.count) # return the most frequent class
   
# Create child splits for a node or make terminal
def split(node, max_depth, min_size, depth):
	#print("now in split")
	left, right = node['groups']
	del(node['groups'])
	# check for a no split
	if not left or not right:
		node['left'] = node['right'] = to_terminal(left + right)
		return
	# check for max depth
	if depth >= max_depth:
		node['left'], node['right'] = to_terminal(left), to_terminal(right)
		return
	# process left child
	if len(left) <= min_size:
		node['left'] = to_terminal(left)
	else:
		node['left'] = get_split(left)
		split(node['left'], max_depth, min_size, depth+1)
	# process right child
	if len(right) <= min_size:
		node['right'] = to_terminal(right)
	else:
		node['right'] = get_split(right)
		split(node['right'], max_depth, min_size, depth+1)
    
# Build a decision tree
def build_tree(train, max_depth, min_size):
  root = get_split(train)
  split(root, max_depth, min_size, 1)
  return root
    
# Make a prediction with a decision tree
def predict_(node, row):
	if row[node['index']] < node['value']: # check if its value is smaller than the node value
		if isinstance(node['left'], dict): # check whether left node is dict
			return predict_(node['left'], row)
		else:
			return node['left']
	else:
		if isinstance(node['right'], dict):
			return predict_(node['right'], row)
		else:
			return node['right']

In [7]:
# @end

###########################
# DO NOT CHANGE THIS CELL #
###########################

df = load_dataset('https://lab.djosix.com/weather.pkl')
X_train, y_train = get_input_target(df)

df.head() # head() means to print out

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Cloud9am,Cloud3pm,Temp9am,RainToday,RainTomorrow
0,7.4,25.100000,0.0,5.469824,7.624853,44.0,4.0,22.0,44.0,25.0,1010.599976,4.43719,4.503167,17.200001,0.0,0
1,12.9,25.700001,0.0,5.469824,7.624853,46.0,19.0,26.0,38.0,30.0,1007.599976,4.43719,2.000000,21.000000,0.0,0
2,9.2,28.000000,0.0,5.469824,7.624853,24.0,11.0,9.0,45.0,16.0,1017.599976,4.43719,4.503167,18.100000,0.0,0
3,17.5,32.299999,1.0,5.469824,7.624853,41.0,7.0,20.0,82.0,33.0,1010.799988,7.00000,8.000000,17.799999,0.0,0
4,14.6,29.700001,0.2,5.469824,7.624853,56.0,19.0,24.0,55.0,23.0,1009.200012,4.43719,4.503167,20.600000,0.0,0


In [8]:
###########################
# DO NOT CHANGE THIS CELL #
###########################

create_model = lambda: Model(X_train.shape[1], 2) # same as clf = DecisionTreeClassifier(), create_model = clf & Model = DecisionTreeClassifier
k_fold_cv(create_model, X_train, y_train).groupby('split').mean()

Run 1:
   train: f1=0.4822 acc=0.8194
     val: f1=0.5253 acc=0.8306
Run 2:
   train: f1=0.4462 acc=0.8248
     val: f1=0.4973 acc=0.8157
Run 3:
   train: f1=0.5125 acc=0.8211
     val: f1=0.4991 acc=0.8098
Run 4:
   train: f1=0.5130 acc=0.8203
     val: f1=0.4967 acc=0.8132
Run 5:
   train: f1=0.5174 acc=0.8137
     val: f1=0.4703 acc=0.8396


,f1,acc
split,,
train,0.494267,0.819842
val,0.497746,0.821784


## Submission

1. Make sure your code runs correctly after clicking `"Runtime" > "Restart and run all"`
2. Rename this notebook to `XXXXXXX_2.ipynb`, where `XXXXXXX` is your student ID.
3. Download IPython notebook: `"File" > "Download" > "Download .ipynb"`
4. Download Python source code: `"File" > "Download" > "Download .py"`
5. Create a zip file for `XXXXXXX_2.ipynb` and `XXXXXXX_2.py`  
   named `XXXXXXX_2.zip`, where `XXXXXXX` is your student ID.
6. Upload the zip file to E3.

😊 Good luck!